In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import json
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint

2024-05-16 18:58:37.419956: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-16 18:58:38.317607: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-16 18:58:38.323633: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 18:58:41.732718: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


# 1.Create model

In [2]:
# Define the CNN model
def create_cnn_model(input_shape):
    model = models.Sequential()

    # Add Convolutional layers
    model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation='relu'))

    # Add Dense layers
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2))  # 2 output neurons for linear and angular speed

    return model

# Create an instance of the model
input_shape = (200, 200, 1)  # Define the input shape based on your camera image dimensions
model = create_cnn_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])  # Using Mean Squared Error as loss for regression

# Print model summary
model.summary()





Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        36928     
                                                                 
 flatten (Flatten)           (None, 135424)            0

# 2. Load data

In [33]:

def load_json(path):

    with open(path, 'r') as f:
        data = json.load(f)

    images = []
    labels = []
    for sample in data:
        try:
            image = cv2.imread(sample['path'], cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (input_shape[0], input_shape[1]))
            image = image / 255.0
            images.append(image)
            labels.append([np.float32(sample['V']), np.float32(sample['W'])])
        except Exception as e:
            print(e)
    
    images = np.array(images, dtype=np.float32)
    labels = np.array(labels, dtype=np.float32)
    return images, labels

images = []
labels = []
paths = ['data/dataset1.json', 'data/dataset2.json']
for path in paths:
    aux_data = load_json(path)
    print(len(aux_data[0]))
    images = np.append(images, aux_data[0])
    labels = np.append(labels, aux_data[1])

[print(x, '->', x.shape) if x.shape!=(200,200) else "" for x in images]
[print(x, '->', len(x)) if len(x)!=2 else "" for x in labels]

images = np.array(images, dtype=np.float32)
labels = np.array(labels, dtype=np.float32)

train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size=0.2, random_state=42)

print("Training images shape:", train_images.shape)
print("Training labels shape:", train_labels.shape)
print("Validation images shape:", val_images.shape)
print("Validation labels shape:", val_labels.shape)



1601


# 3. Train

In [8]:

model = create_cnn_model(input_shape)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Define callbacks (optional but useful for saving the best model)
checkpoint_path = "best_model.h5"
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# Train the model
batch_size = 32
epochs = 10

history = model.fit(train_images, train_labels,
                    validation_data=(val_images, val_labels),
                    batch_size=batch_size,
                    epochs=epochs,
                    callbacks=[checkpoint])

# Evaluate the model on the validation set
loss, mae = model.evaluate(val_images, val_labels)
print("Validation Loss:", loss)
print("Validation Mean Absolute Error:", mae)

# Load the best model
best_model = tf.keras.models.load_model(checkpoint_path)

# Make predictions on the validation set using the best model
predictions = best_model.predict(val_images)

# Optionally, you can save the best model for later use
best_model.save("data/models/best_model_trained.h5")

2024-05-15 09:35:40.818941: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 190720000 exceeds 10% of free system memory.


Epoch 1/10


2024-05-15 09:35:44.115760: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 160579584 exceeds 10% of free system memory.
2024-05-15 09:35:45.850804: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 160579584 exceeds 10% of free system memory.


 1/38 [..............................] - ETA: 3:19 - loss: 17.3316 - mae: 2.9767

2024-05-15 09:35:46.849692: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 160579584 exceeds 10% of free system memory.
2024-05-15 09:35:48.309789: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 160579584 exceeds 10% of free system memory.


38/38 [==============================] - ETA: 0s - loss: 4.8423 - mae: 1.0914
Epoch 1: val_loss improved from inf to 0.23703, saving model to best_model.h5


/home/miquel/.local/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


38/38 [==============================] - 98s 3s/step - loss: 4.8423 - mae: 1.0914 - val_loss: 0.2370 - val_mae: 0.2532
Epoch 2/10
38/38 [==============================] - ETA: 0s - loss: 0.1548 - mae: 0.1974
Epoch 2: val_loss improved from 0.23703 to 0.08623, saving model to best_model.h5
38/38 [==============================] - 93s 2s/step - loss: 0.1548 - mae: 0.1974 - val_loss: 0.0862 - val_mae: 0.1316
Epoch 3/10
38/38 [==============================] - ETA: 0s - loss: 0.0857 - mae: 0.1410
Epoch 3: val_loss did not improve from 0.08623
38/38 [==============================] - 95s 2s/step - loss: 0.0857 - mae: 0.1410 - val_loss: 0.0984 - val_mae: 0.2036
Epoch 4/10
38/38 [==============================] - ETA: 0s - loss: 0.0762 - mae: 0.1481
Epoch 4: val_loss improved from 0.08623 to 0.07072, saving model to best_model.h5
38/38 [==============================] - 93s 2s/step - loss: 0.0762 - mae: 0.1481 - val_loss: 0.0707 - val_mae: 0.1217
Epoch 5/10
38/38 [============================